In [1]:
import Python
import Dispatch
import TensorFlow
let pd = Python.import("pandas")
let np = Python.import("numpy")

# Library

In [2]:
extension String {
    func strip() -> Substring {
        return self.drop{$0==" "}
    }
}

// Time how long it takes to run the specified function, optionally taking
// the average across a number of repetitions.
public func time(repeating: Int = 1, _ function: () -> ()) {
    guard repeating > 0 else { return }
    
    // Warmup
    if repeating>1 {function()}
    
    var times = [Double]()
    for _ in 1...repeating {
        let start = DispatchTime.now()
        function()
        let end = DispatchTime.now()
        let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
        let milliseconds = nanoseconds / 1e6
        times.append(milliseconds)
    }
    print("average: \(times.reduce(0.0, +)/Double(times.count)) ms,   " +
          "min: \(times.reduce(times[0], min)) ms,   " +
          "max: \(times.reduce(times[0], max)) ms")
}


In [22]:
public typealias TF=Tensor<Float>

public struct Individual{
    public var data: TF
    public var fitnessValue: Float?=nil
    public var constraints: TF? = nil
    public var constraintsSum: Float? = nil
    public var isFeasible: Bool? = nil
    
    public init(data: TF) {
        self.data = data
    }
    
    public init(random d: Int, lowerLimit l: Float = -5, upperLimit u: Float = 5) {
        self.init(data: Tensor<Float>(randomUniform: [d]) * (u-l) + l)
    }   
}

public struct Population {
    public var individuals: [Individual]
    
    public init(individuals: [Individual]) {
        self.individuals = individuals
    }

    public init(random n: Int, dimensions d: Int, lowerLimit l: Float = -5, upperLimit u: Float = 5) {
        var res: [Individual] = []
        for _ in 0..<n { res.append(Individual(random: d, lowerLimit: l, upperLimit: u)) }        
        self.init(individuals: res)
    }
}

public protocol ConstraintParam {
    associatedtype T
    var data: T { get }
    var maxTime: Int { get }
}

extension ConstraintParam where T==TF {
    public var maxTime: Int {
        return data.shape[0]
    }
}

public struct SingleContraintParam: ConstraintParam {
    public var data: TF
    
    public init(_ data: TF) {
        assert(data.shape.count == 1)
        self.data = data
    }
    public init(_ data: [Float]) {
        self.init(TF(data))
    }
}

public struct MultiContraintParams: ConstraintParam {
    public var data: TF
    
    public init(_ data: TF) {
        self.data = data
    }
}

final public class Optimization<C: ConstraintParam> {
    public typealias FitnessFn=((Individual) -> TF)
    public typealias ConstraintFun=((Individual,Float) -> TF)
    
    public var population: Population
    public var getFitness: FitnessFn
    public var getConstraints: ConstraintFun
    public let constraintParams: C
    public let frequency: Int
    public let maxTime: Int
    public let CR: Float = 0.3
    public let betaMin: Float = 0.2
    public let betaMax: Float = 0.8
    public let maxEvals: Int? = nil
    
    public init(population: Population, getFitness: @escaping FitnessFn, getConstraints: @escaping ConstraintFun,
                constraintParams: C, frequency: Int) {
        self.population = population
        self.getFitness = getFitness
        self.getConstraints = getConstraints
        self.constraintParams = constraintParams
        self.frequency = frequency
        self.maxTime = constraintParams.maxTime
    }
    
}

extension Optimization {
    public func run(_ generations: Int) {
        do{
            for i in 0..<generations {
                print(i)
            }
        }
    }
}

In [23]:
time(repeating: 100, { Population(random: 100, dimensions: 1_000) })

average: 12.400186809999996 ms,   min: 9.922771 ms,   max: 17.788368 ms


# Use

In [24]:
var df = pd.read_csv("/home/renato/github/DENN/data/medium/dC_01.csv")
let ab = SingleContraintParam(df.columns.map{Float(String($0)!.strip())!})

In [25]:
let D = 30
let frequency = 1_000
let totalGenerations = ab.maxTime * frequency + 1000

In [26]:
func getFitness(_ indv: Individual) -> TF {
    return pow(indv.data, 2).sum()
}

func getConstraint(_ indiv: Individual, _ b: Float) -> TF {
    return (sqrt(Float(D)) * indiv.data).sum() - b
}

In [27]:
var population = Population(random: 20, dimensions: D)

In [28]:
var opt = Optimization(population: population, getFitness: getFitness, getConstraints: getConstraint,
                       constraintParams: ab, frequency: frequency)

In [29]:
opt.maxTime

100


In [32]:
opt.run(10)

0
1
2
3
4
5
6
7
8
9
